In [81]:
from pymongo import MongoClient
import os
import pandas as pd
from neo4j import GraphDatabase

In [82]:
mongo = MongoClient(os.getenv("MONGO_URI", "mongodb://localhost:27017"))

In [83]:
mdb = mongo["backoffice"]

In [84]:
df_prov = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
df_tel  = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
df_prod = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
df_op   = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
df_det  = pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")

In [85]:
df_prov["CUIT_proveedor"] = df_prov["CUIT_proveedor"].astype(str)


In [86]:
def is_not_all_nan(phone):
    import math
    return not all(
        (isinstance(phone.get(f), float) and math.isnan(phone.get(f))) or phone.get(f) is None
        for f in ['codigo_area', 'nro_telefono', 'tipo']
    )

res = pd.merge(df_prov,df_tel, on="id_proveedor",how="left")
proveedores = res.groupby("id_proveedor").apply(lambda g: {
    'id_proveedor': int(g['id_proveedor'].iloc[0]),
    "CUIT_proveedor": g["CUIT_proveedor"].iloc[0],
    "razon_social":g["razon_social"].iloc[0],
    "tipo_sociedad":g["tipo_sociedad"].iloc[0] ,
    "direccion":g["direccion"].iloc[0]  , 
    "activo" :int(g["activo"].iloc[0]),
    "habilitado":int(g["habilitado"].iloc[0]),
    "telefonos": [t for t in g[['codigo_area', 'nro_telefono', 'tipo']].to_dict(orient='records') if is_not_all_nan(t)]
})

proveedores = proveedores.reset_index(name='document')["document"].to_list()
proveedores

C:\Users\ianda\AppData\Local\Temp\ipykernel_27136\1664017898.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  proveedores = res.groupby("id_proveedor").apply(lambda g: {


[{'id_proveedor': 1,
  'CUIT_proveedor': '30660608172',
  'razon_social': 'Arcos Plateados',
  'tipo_sociedad': 'SA',
  'direccion': 'Pelliza 4234, Olivos, BA',
  'activo': 0,
  'habilitado': 0,
  'telefonos': [{'codigo_area': 513.0, 'nro_telefono': 4998612.0, 'tipo': 'M'},
   {'codigo_area': 992.0, 'nro_telefono': 4241515.0, 'tipo': 'F'}]},
 {'id_proveedor': 2,
  'CUIT_proveedor': '30660608173',
  'razon_social': 'Grupo Fernández',
  'tipo_sociedad': 'SRL',
  'direccion': 'San Martín 640 Piso 7, CABA',
  'activo': 1,
  'habilitado': 1,
  'telefonos': [{'codigo_area': 638.0,
    'nro_telefono': 4819478.0,
    'tipo': 'F'}]},
 {'id_proveedor': 3,
  'CUIT_proveedor': '30660608174',
  'razon_social': 'Innovation',
  'tipo_sociedad': 'SAS',
  'direccion': 'Av del Libertador 32560, CABA',
  'activo': 1,
  'habilitado': 1,
  'telefonos': [{'codigo_area': 915.0,
    'nro_telefono': 4686947.0,
    'tipo': 'F'}]},
 {'id_proveedor': 4,
  'CUIT_proveedor': '30660608175',
  'razon_social': 'Alimen

In [87]:
res = pd.merge(df_op,df_det,on="id_pedido",how="outer")
ordenes = res.groupby('id_pedido').apply(lambda g: {
    "id_pedido": int(g['id_pedido'].iloc[0]),
    'id_proveedor': int(g['id_proveedor'].iloc[0]),
    'fecha': g['fecha'].iloc[0],
    'iva': float(g['iva'].iloc[0]),
    'items': g[['id_producto', 'nro_item', 'cantidad']].to_dict(orient='records')
}).reset_index(name='document')["document"].to_list()

C:\Users\ianda\AppData\Local\Temp\ipykernel_27136\2708529387.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ordenes = res.groupby('id_pedido').apply(lambda g: {


In [88]:
import math

def replace_nans(obj):
    if isinstance(obj, float):
        if math.isnan(obj) or math.isinf(obj):
            return None
        return obj
    elif isinstance(obj, dict):
        return {k: replace_nans(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_nans(x) for x in obj]
    else:
        return obj

def import_mongo():
    mdb.proveedores.delete_many({})
    mdb.proveedores.insert_many(replace_nans(proveedores))

    mdb.productos.delete_many({})
    mdb.productos.insert_many(replace_nans(df_prod.to_dict("records")))

    mdb.ordenes.delete_many({})
    mdb.ordenes.insert_many(replace_nans(ordenes))

import_mongo()

In [89]:
uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
neo = GraphDatabase.driver(uri, auth=("neo4j", os.getenv("NEO4J_PASSWORD", "epicpassword123")))

In [90]:
def import_neo4j():
    with neo.session() as sess:
        sess.run("MATCH (n) DETACH DELETE n")
        # proveedores
        df = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (p:Proveedor {
                  id_proveedor:$id_proveedor,
                  cuit: $CUIT_proveedor,
                  razon_social: $razon_social,
                  activo: $activo,
                  habilitado: $habilitado
                })
            """, **r.to_dict())

        # RELACIÓN TELEFONO
        df = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                CREATE (p)-[:TIENE_TELEFONO {numero:$telefono}]->(:Telefono)
            """, id_proveedor=r.id_proveedor, telefono=r.nro_telefono)

        # productos
        df = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (:Producto {
                  codigo: $id_producto,
                  nombre: $descripcion,
                  marca: $marca,
                  precio: $precio
                })
            """, **r.to_dict())

        # ordenes y detalles
        df_op = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
        df_op["fecha"] = pd.to_datetime(df_op["fecha"], dayfirst=True).dt.strftime("%Y-%m-%d")
        df_det= pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")
        
        for _,o in df_op.iterrows():
            sess.run("""
                CREATE (o:Orden {
                     id_pedido:$id_pedido,
                     id_proveedor:$id_proveedor,
                     iva:$iva,
                  fecha: date($fecha)
                })
                WITH o
                MATCH (p:Proveedor {id_proveedor:$id_proveedor})
                CREATE (o) - [:LE_COMPRA_A] -> (p)
            """, **o.to_dict())

        #Orden tiene (3 de este) producto
        
        for _,detalle in df_det.iterrows():
            sess.run("""
                MATCH (o:Orden {id_pedido:$id_pedido})
                MATCH (p:Producto {codigo: $id_producto})
                CREATE (o)-[:REALIZA_PEDIDO {cantidad: $cantidad}] -> (p)
            """, **detalle.to_dict())


        for _,d in df_det.iterrows():
            sess.run("""
                MATCH (o:Orden {id_pedido:$id_pedido,id_producto:$id_producto}), (pr:Producto {codigo:$id_producto})
                CREATE (o)-[:CONTiene {cantidad:$cantidad}]->(pr)
            """, **d.to_dict())
import_neo4j()